In [28]:
import pandas as pd
import math
from iapws import IAPWS97
import sys
%run equations_module.ipynb
%run class_module.ipynb

0.002868007145475173
2759.5950361387345 6.732103638809531
306.3233808143922 14274.669545950675
2600.4304108866704 7.986839903989945 328.3328322561139
steam 2777.1195376846617 6.58497899635217 453.0356323914666
brine 762.6828443354106 2.138431350899127 453.0356323914666


In [29]:
#import data frame for pipe network;
net_df = pd.read_excel('fcrs_in.xls', sheetname=2)#2
#import data frame for NODE attributes
node_df = pd.read_excel('fcrs_in.xls', sheetname=0)#0
#import dataframe for RI pipeline attributes
pipe_df = pd.read_excel('fcrs_in.xls', sheetname=1)#1
#node_df.head()

In [30]:
network_graph = {}
node_dict = {}
pipe_dict = {}
#Create a graph from nodes; in dictionary format
for nodes in range (len(net_df.index)):
    nodeID, inlet, outlet, pflow = map(lambda x: net_df.iloc[nodes, x], range(0,4))
    #print(nodeID, inlet, outlet, pflow)
    network_graph[nodeID] = [inlet, outlet, pflow]

for nodes in range(len(node_df.index)):
    #create a node class for each row and store it in a dictionary
    nodeID,b,c,d,e,f,g,h,i,j,k = map(lambda x: node_df.iloc[nodes, x], range(0,11))
    node_dict[nodeID] = node_class(nodeID,b,c,d,e,f,g,h,i,j,k)

for pipelines in range(len(pipe_df.index)):
    #create a pipeline class for each row and store it in a dictionary
    lineID,b,c,d,e,f,g,h,i,j,k  = map(lambda x: pipe_df.iloc[pipelines, x], range(0,11))   
    pipe_dict[lineID] = pipe_class(lineID,b,c,d,e,f,g,h,i,j,k)

In [31]:
#[print(net, network_graph[net]) for net in sorted(network_graph)]
#[print(node_dict[net]) for net in sorted(node_dict)]
#[print(pipe_dict[net]) for net in sorted(pipe_dict)]

In [32]:
def print_f (node1, node2, pipe):
    print('{:14} {:>14} {:>14}'.format('', 'Inlet Node', 'Outlet Node'))
    print('{:14} {:14.4f} {:14.4f}'.format('Pressure', node1.p, node2.p))
    print('{:14} {:14.4f} {:14.4f}'.format('Temperature', node1.t, node2.t))
    print('{:14} {:14.4f} {:14.4f}'.format('Mass Flow', node1.mf, node2.mf))
    print('{:14} {:14.4f} {:14.4f}'.format('Diameter', node1.d, node2.d))
    print('{:14} {:14.4f} {:14.4f}'.format('Velocity', node1.v, node2.v))
    print('{:14} {:14.4f}'.format('Reynolds Number', node1.reynolds))
    print('{:14} {:14.4f} {:14.4f}'.format('Enthalpy', node1.h, node2.h))
    print('{:14} {:14.4f} {:14.4f}'.format('SpecificExergy', node1.sp_exergy, node2.sp_exergy))
    print('{:14} {:14.4f} {:14.4f}'.format('Exergy', node1.exergy, node2.exergy))
    
    #pipeline
    print('\n{}'.format('Pipeline Section'))
    print('{:18} {:14.4f}'.format('Length', pipe.length))
    print('{:18} {:14.4f}'.format('Total F_Loss', pipe.f_tot))
    print('{:18} {:14.4f}'.format('  Skin Friction', pipe.f_skin))
    print('{:18} {:14.4f}'.format('  Fittings Loss', pipe.f_fit))
    print('{:18} {:14.4f}'.format('  Expansion', pipe.f_exp))
    print('{:18} {:14.4f}'.format('  Contraction', pipe.f_con))

    #print('{:18} {:14.4f}'.format('h_out (J/s)', pipe.h_out))
    print('{:18} {:14.4f}'.format('Heat Loss (J/kg)', pipe.Q))
    print('{:18} {:14.4f}'.format('gdz', pipe.gdz))
    print('{:18} {:14.4f}'.format('udu', pipe.udu))
    print('{:18} {:14.4f}'.format('Total Loss', pipe.t_loss))
    print('\n')


In [33]:
'''Main Function to calculate Pressure drop along the steamline'''
def calc_dp (node_in, node_out, pipeline, p_f):
    #print(node_dict['S501'],'\n',node_dict['S501X'],'\n',pipe_dict['STL501'])
    print (pipeline)
    
    '''Calculate Steam T_sat, Rho and Mu from steam table'''
    node_dict[node_in].steam_p()
    #print(node_dict[node_in].t)
   
    '''Calculate area and velocity of inlet node; Only area for outlet node'''
    node_dict[node_in].calc_area()
    node_dict[node_in].calc_velocity(node_dict[node_in])
    node_dict[node_out].calc_area()
    node_dict[node_out].calc_velocity(node_dict[node_in])
    #print('{:.4f}'.format(node_dict[node_in].area))
    
    '''Calculate Relative Roughness of the pipeline'''
    pipe_dict[pipeline].rel_roughness(node_dict[node_in].d)
    #print('{:.8f}'.format(pipe_dict[pipeline].rel_rough))
    
    '''Calculate Reynolds Number at the inlet node'''
    node_dict[node_in].nRE()
    #print('{:.0f}'.format(node_dict[node_in].reynolds))
    
    '''Calculate friction factor of the pipe'''
    if node_dict[node_in].reynolds > 0: 
        pipe_dict[pipeline].ff_fanning(node_dict[node_in].reynolds)
       #print('{}'.format(pipe_dict[pipeline].ff))
    else:
        pass
   
    '''Calculate skin friction along the pipe'''
    if node_dict[node_in].reynolds > 0: 
        pipe_dict[pipeline].skin_friction(node_dict[node_in].v, node_dict[node_in].d)
        #print(pipe_dict[pipeline].ff, pipe_dict[pipeline].length, node_dict[node_in].v, node_dict[node_in].d)
        #print('{}'.format(pipe_dict[pipeline].f_skin))
    else:
        pipe_dict[pipeline].f_skin = 0
    
    '''Calculate expansion or contraction friction loss'''
    if node_dict[node_in].area == node_dict[node_out].area:
        pipe_dict[pipeline].f_exp = 0
        pipe_dict[pipeline].f_con = 0
    
    if node_dict[node_in].area < node_dict[node_out].area:
        pipe_dict[pipeline].f_con = 0
        pipe_dict[pipeline].f_expansion(node_dict[node_in].area, node_dict[node_out].area, node_dict[node_in].v)
    
    else:
        pipe_dict[pipeline].f_exp = 0
        pipe_dict[pipeline].f_contraction(node_dict[node_in].area, node_dict[node_out].area, node_dict[node_in].v)
    
    '''Calculate Friction Loss from fittings'''
    pipe_dict[pipeline].f_fittings(kf_list, node_dict[node_in].v)
    
    '''Calcutate total friction loss'''
    pipe_dict[pipeline].f_total()
    
    '''Calculate Heat Loss'''
    pipe_dict[pipeline].q_loss(node_dict[node_in], insul)
    
    '''Calculate gdz and udu'''
    pipe_dict[pipeline].f_gdz(node_dict[node_in].z, node_dict[node_out].z)
    pipe_dict[pipeline].f_udu(node_dict[node_in].v, node_dict[node_out].v)
    
    '''Calculate Outlet Pressure'''
    node_dict[node_out].p_out(node_dict[node_in], pipe_dict[pipeline])
    #print('P-out = ', node_dict[node_in].p, node_dict[node_out].p)
        
    '''Total Loss'''
    pipe_dict[pipeline].total_loss()
    
    '''Calculate mass Flow of Outlet Node'''
    node_dict[node_out].calc_mf(node_dict[node_in], pipe_dict[pipeline], p_f)
    
    '''Calculate Exergy at Inlet and Outlet'''
    node_dict[node_out].calc_exergy()
    node_dict[node_in].calc_exergy()
    
    '''Print Node and Pipe Parameters'''
    #print_f(node_dict[node_in], node_dict[node_out], pipe_dict[pipeline])

In [34]:
'''Create instance of Kf for fittings and insulation params'''
kf_list = kf()
insul = insulation()


for x in sorted(network_graph):
    ''' x is the network graph key which is also the pipeID
    pipe_dict[x] -> pipeline
    network_graph[x][0] -> Inlet Node
    network_graph[x][1] -> Outlet Node
    '''
    '''Calculate Steam T_sat, Rho and Mu from steam table'''
    #print(pipe_dict[x], network_graph[x][2])
    calc_dp(network_graph[x][0], network_graph[x][1], x, network_graph[x][2])
    

#for node in sorted(node_dict):
#    print(node_dict[node].ID, node_dict[node].exergy, node_dict[node].mf, node_dict[node].p)
#
#for pipe in sorted(pipe_dict):
#    print(pipe_dict[pipe].ID, pipe_dict[pipe].Q, pipe_dict[pipe].f_tot, pipe_dict[pipe].f_skin, pipe_dict[pipe].f_fit,\
#         pipe_dict[pipe].udu, pipe_dict[pipe].gdz)

STL501
STL501X70A1
STL501X70A2
STL502
STL502X70A1
STL502X70A2
STL503
STL503X70A1
STL503X70A2
STL504
STL504X70A1
STL504X70A2
STL505
STL505X70A1
STL505X70A2
STL506
STL506X70B1
STL506X70B2
STL508
STL508X70C1
STL508X70C2
STL70A1
STL70A2
STL70B1
STL70B2
STL70IF1
STL70IF2


In [35]:
#def f_fittings (nf, kf, v):
#    v_l = np.full(8,v)
#    f_f = nf * kf * v**2 * 0.5
#    self.f_fit = sum(f_f)
#f_fittings(nf, kf_list, )